In [ ]:
import openreview, time, datetime

client = openreview.api.OpenReviewClient(
    username='',
    password='',
    baseurl='https://devapi2.openreview.net'
)

# Open Invitations

In [ ]:
def open_invitations(
        venue_id,
        paper_numbers_info
):
    """
    Opens the Official_Comment invitation for the specified paper numbers
    and re-opens/un-expires the Review_Issue_Report invitation for each review if needed.

    paper_numbers_info is a dict mapping from paper number to a dict with keys:
        - 'close_author_response_date'
        - 'close_reviewer_response_date'
        - 'close_review_issue_date'
    These dates should be in milliseconds since epoch. For example, to convert a datetime to milliseconds, you can use:

    date = openreview.tools.datetime_millis(
        datetime.datetime(2025, 12, 12, 11, 59, tzinfo=datetime.timezone.utc)
    )
    
    or for relative dates (e.g., 30 days from now):

    date = openreview.tools.datetime_millis(
        datetime.datetime.now() + datetime.timedelta(days=30)
    )

    Args:
        venue_id (str): ID of the current cycle (e.g., "aclweb.org/ACL/ARR/2026/January").
        paper_numbers_info (dict): Mapping from paper number to a dict with key dates.

    Returns:
        None: Logs progress and applies invitation edits via the OpenReview client/API.
    """
    
    domain = client.get_group(venue_id)
    meta_invitation_id = domain.content['meta_invitation_id']['value']

    if len(paper_numbers_info) == 0:
        print("No papers found")
        return
        
    now_millis = openreview.tools.datetime_millis(datetime.datetime.now())

    submissions = client.get_all_notes(
        invitation=f"{venue_id}/-/Submission",
    )
    print(f"Total submissions: {len(submissions)}")

    for num, dates in paper_numbers_info.items():

        close_review_issue_date = dates['close_review_issue_date']

        invitation = openreview.tools.get_invitation(
            client,
            f"{venue_id}/Submission{num}/-/Official_Comment"
        )
        # invitation should exist
        if invitation is None:
            print(f"Invitation for paper number {num} not found, skipping...")
            continue

        current_invitees = invitation.invitees
        current_signatures = invitation.edit['signatures']['param']['items']
        current_readers = invitation.edit['note']['readers']['param']['enum']
        outer_readers = invitation.readers
        
        # Build final lists
        final_invitees, final_signatures, final_readers, final_outer_readers = None, None, None, None
        if not any('Authors' in invitee for invitee in current_invitees):
            final_invitees = current_invitees + [f"{venue_id}/Submission{num}/Authors"]
        if not any('Authors' in reader for reader in current_readers):
            final_readers = current_readers + [f"{venue_id}/Submission{num}/Authors"]
        if not any('Authors' in reader for reader in outer_readers):
            final_outer_readers = outer_readers + [f"{venue_id}/Submission{num}/Authors"]
        signature_values = [item['prefix'] if 'prefix' in item else item['value'] for item in current_signatures]
        if not any('Authors' in signature for signature in signature_values):
            final_signatures = current_signatures + [{'value': f"{venue_id}/Submission{num}/Authors", 'optional': True}]

        # Build the edit and log changes
        edit = {}
        log_str = []
        if final_signatures is not None:
            edit['signatures'] = {
                'param': {
                    'items': final_signatures
                }
            }
            log_str.append('inner signatures')
        if final_readers is not None:
            edit['note'] = {
                'readers': {
                    'param': {
                        'enum': final_readers
                    }
                }
            }
            log_str.append('inner readers')
        if final_invitees is not None:
            log_str.append('invitees')
        if final_outer_readers is not None:
            log_str.append('outer readers')
        

        if final_invitees is not None or final_readers is not None or final_signatures is not None or final_outer_readers is not None:
            print(f"Modifying paper number {num}: {', '.join(log_str)}")
            client.post_invitation_edit(
                invitations=meta_invitation_id,
                readers=[venue_id],
                writers=[venue_id],
                signatures=[venue_id],
                invitation=openreview.api.Invitation(
                    id=invitation.id,
                    signatures=[venue_id],
                    invitees=final_invitees,
                    readers=final_outer_readers,
                    edit=edit
                )
            )

        # Handle review issue
        # (reviewer response)
        for review_number in range(1, 11):
            try:
                invitation = client.get_invitation(
                    id=f"{venue_id}/Submission{num}/Official_Review{review_number}/-/Review_Issue_Report"
                )
            except:
                continue
            cdate, expdate = None, None
            if invitation.cdate > now_millis: # if not activated, open
                cdate = now_millis
            if invitation.expdate != close_review_issue_date: # if expired, unexpire, 
                expdate = close_review_issue_date

            log_str = ''
            if cdate is not None:
                log_str += f"Re-opening Submission{num}/Official_Review{review_number}/-/Review_Issue_Report"
            if expdate is not None:
                log_str += f"\nUn-expiring Submission{num}/Official_Review{review_number}/-/Review_Issue_Report"

            if cdate is not None or expdate is not None:
                print(log_str)
                client.post_invitation_edit(
                    invitations=meta_invitation_id,
                    readers=[venue_id],
                    writers=[venue_id],
                    signatures=[venue_id],
                    invitation=openreview.api.Invitation(
                        id=invitation.id,
                        signatures=[venue_id],
                        cdate=cdate,
                        expdate=expdate
                    )
                )        

# Close Invitations

In [ ]:
def close_invitations(
    venue_id,
    paper_numbers_info,
) -> None:
    """
    Closes the Official_Comment invitation for different groups depending on the dates.
    If the current time is past the close_author_response_date, removes authors from invitees, signatures, and readers.
    If the current time is past the close_reviewer_response_date, removes authors from readers to prevent reviewers from writing comments visible to authors.

    paper_numbers_info is a dict mapping from paper number to a dict with keys:
        - 'close_author_response_date'
        - 'close_reviewer_response_date'
        - 'close_review_issue_date'
    These dates should be in milliseconds since epoch. For example, to convert a datetime to milliseconds, you can use:

    date = openreview.tools.datetime_millis(
        datetime.datetime(2025, 12, 12, 11, 59, tzinfo=datetime.timezone.utc)
    )
    
    or for relative dates (e.g., 30 days from now):

    date = openreview.tools.datetime_millis(
        datetime.datetime.now() + datetime.timedelta(days=30)
    )

    Args:
        venue_id (str): ID of the current cycle (e.g., "aclweb.org/ACL/ARR/2026/January").
        paper_numbers_info (dict): Mapping from paper number to a dict with key dates.

    Returns:
        None: Logs progress and applies invitation edits via the OpenReview client/API.
    """
    
    domain = client.get_group(venue_id)
    meta_invitation_id = domain.content['meta_invitation_id']['value']

    if len(paper_numbers_info) == 0:
        print("No papers found")
        return
    now_millis = openreview.tools.datetime_millis(datetime.datetime.now())

    submissions = client.get_all_notes(
        invitation=f"{venue_id}/-/Submission",
    )
    print(f"Total submissions: {len(submissions)}")

    for num, dates in paper_numbers_info.items():
        close_author_response_date = dates['close_author_response_date']
        close_reviewer_response_date = dates['close_reviewer_response_date']
        try:
            invitation = client.get_invitation(
                id=f"{venue_id}/Submission{num}/-/Official_Comment"
            )
        except:
            print(f"Invitation for paper number {num} not found, skipping...")
            continue

        current_invitees = invitation.invitees
        outer_readers = invitation.readers
        current_signatures = invitation.edit['signatures']['param']['items']
        current_readers = invitation.edit['note']['readers']['param']['enum']

        final_invitees, final_signatures, final_readers, final_outer_readers = None, None, None, None
        if any('Authors' in invitee for invitee in current_invitees) and now_millis > close_author_response_date:
            final_invitees = [
                invitee for invitee in current_invitees if 'Authors' not in invitee
            ]
        if any('Authors' in reader for reader in outer_readers) and now_millis > close_author_response_date:
            final_outer_readers = [
                reader for reader in outer_readers if 'Authors' not in reader
            ]
        if any('Authors' in reader for reader in current_readers) and now_millis > close_reviewer_response_date:
            final_readers = [
                reader for reader in current_readers if 'Authors' not in reader
            ]
        signature_values = [item['prefix'] if 'prefix' in item else item['value'] for item in current_signatures]
        if any('Authors' in signature for signature in signature_values) and now_millis > close_author_response_date:
            final_signatures = [
                signature_obj for signature_obj in current_signatures
                if 'Authors' not in (signature_obj.get('prefix', '') if 'prefix' in signature_obj else signature_obj.get('value', ''))
            ]

        # Build the edit and log changes
        edit = {}
        log_str = []
        if final_signatures is not None:
            edit['signatures'] = {
                'param': {
                    'items': final_signatures
                }
            }
            log_str.append('inner signatures')
        if final_readers is not None:
            edit['note'] = {
                'readers': {
                    'param': {
                        'enum': final_readers
                    }
                }
            }
            log_str.append('inner readers')
        if final_invitees is not None:
            log_str.append('invitees')
        if final_outer_readers is not None:
            log_str.append('outer readers')

        if final_invitees is not None or final_readers is not None or final_signatures is not None or final_outer_readers is not None:
            print(f"Modifying paper number {num}: {', '.join(log_str)}")
            client.post_invitation_edit(
                invitations=meta_invitation_id,
                readers=[venue_id],
                writers=[venue_id],
                signatures=[venue_id],
                invitation=openreview.api.Invitation(
                    id=invitation.id,
                    signatures=[venue_id],
                    invitees=final_invitees,
                    readers=final_outer_readers,
                    edit=edit
                )
            )
        

In [ ]:
paper_numbers_info = {
    1: {
        'close_author_response_date': openreview.tools.datetime_millis(datetime.datetime.now() + datetime.timedelta(seconds=30)),
        'close_reviewer_response_date': openreview.tools.datetime_millis(datetime.datetime.now() + datetime.timedelta(seconds=60)),
        'close_review_issue_date': openreview.tools.datetime_millis(datetime.datetime.now() + datetime.timedelta(days=30))
    }
}

In [ ]:
open_invitations(
    venue_id=venue.id,
    paper_numbers_info=paper_numbers_info
)

In [ ]:
close_invitations(
    venue_id=venue.id,
    paper_numbers_info=paper_numbers_info
)